# Step 1: Data Pre-processing
This notebook is for the initial set up of the LLM application that will be the same for each model.

The focus is on the data-pre processing step of the RAG pipeline and getting the data into the vector database.

We need to put the data of the SMU Catalog of 2023-2024 into Qdrant which is a cloud vector database. This will allow the language model to access and retrieve the necessary information.

There are many changes that can be done at this step to alter how the text goes into the vector database (ex: different text splitters, document loaders, retrievers, etc.)

We will install some of the necessary dependancies now and the rest along the way throughout the notebook.

In [ ]:
%pip install openai pypdf qdrant-client langchain python-dotenv tiktoken langchain-openai pandas

In [ ]:
# Set up to initialize API keys from .env file into the
import os
from dotenv import find_dotenv, load_dotenv

# Load environment variables from the .env files
load_dotenv(find_dotenv(filename='SURF-Project_Optimizing-PerunaBot/setup/.env'))

Here we will initialize langsmith for tracing and tracking

In [ ]:
from langsmith import Client
langchain_api_key = os.environ["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"]
langchain_endpoint = os.environ["LANGCHAIN_ENDPOINT"]
langsmith_project = os.environ["LANGCHAIN_PROJECT"]

langsmith_client = Client()

## PDF Breakdown
If you take a look at the pdfs in the data folder, just the catalog pdf alone is over 800 pages long! To upload it into the vector databse, we have to 1st get all the text from the documents.

First we will do the PDFs, then later on the CSV of the FAQs with a different method.

In [ ]:
# langchain imports
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader

# file paths to the two PDFs we're using
pdf_paths = ['C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/20232024 Undergraduate Catalog91123.pdf',
             'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/Official University Calendar 2023-2024.pdf',
             'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/2023_PerunaPassport.pdf',
             'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/SMU Student Handbook 23-24.pdf',
             'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/SMUCampusGuideFactsMap.pdf'
             ]

def load_pdfs_with_langchain(pdf_paths):
    documents = []
    for path in pdf_paths:
        try:
            # Use LangChain's PyPDFLoader to load the PDF
            loader = PyPDFLoader(path)
            # Load and pase the PDF into document instances
            pdf_doc = loader.load()
            # Insert pdf into documents list variable
            documents.extend(pdf_doc)
        except Exception as e:
            print(f"Error loading {path}: {e}")
    return documents

#Load PDF documents using the function
docs = load_pdfs_with_langchain(pdf_paths)

print(len(docs))
print(docs[0].page_content[0:100])
print(docs[0].metadata)

Here we are initializing an API connection to the Qdrant vector database from https://qdrant.tech/

In [3]:
from qdrant_client import qdrant_client
from qdrant_client.http import models

In [4]:
# Initializing Qdrant host URL and API key
qdrant_host = os.environ['QDRANT_HOST']
qdrant_api_key = os.environ['QDRANT_API_KEY']

#Initialize Qdrant Client
client = qdrant_client.QdrantClient(
    url=qdrant_host, 
    api_key = qdrant_api_key,
)

Now we are going to create the 1st collection of vectors and the vectorstore inside the database. Eventually, we will have more than one collection to see how changes to how the data is uploaded and retrieved affects the accuracy and other evaluation metrics.

This is a very important step because an LLM application is only as good as its data and the documents it retrieves to create an answer.

Since we will have more than one collection within the vector database, we will just create a function that will allow us to create a new vectorstore collection when needed.

In [5]:
# function to create a vector store based on the collection name
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import Qdrant

# Initializing OpenAI API key for embeddings and later use
openai_api_key = os.environ['OPENAI_API_KEY']

def create_vectorstore(qdrant_collection_name):
    
    vectors_config = models.VectorParams(
   size=1536, #for OpenAI
   distance=models.Distance.COSINE
   )
    
    client.create_collection(
   collection_name = qdrant_collection_name,
   vectors_config=vectors_config,   
    )

    vector_store = Qdrant(
        client=client, 
        collection_name=qdrant_collection_name, 
        embeddings=OpenAIEmbeddings(),
    )
  
    return vector_store

In [8]:
# create 1st collection of vectors

qdrant_collection_1 = os.environ['QDRANT_COLLECTION_1']
vector_store_1 = create_vectorstore(qdrant_collection_1)

This is where the experiment begins! Now we have all the text from the pdfs in the pdfs_doc_text variable, the text needs to be split into chunks using langchain text splitters to be turned into vectors using the OpenAI Embeddings Model.

We are going to test two methods:
1. Parent Doucment Retriever method with the **RecursiveCharacterTextSplitter**

2. Semantic Chunking method using the **Semantic Text Splitter**

Since this will generate two different types of chunks, we will but them in two different collections within the vector database.

In [ ]:
# Parent Document Retriever Method
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

child_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=25, 
                                                length_function=len, add_start_index=True) 

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=50, 
                                                length_function=len, add_start_index=True)  

# storage for parent splitter
store = InMemoryStore()

# retriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=vector_store_1, 
    docstore=store, 
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    )
# adding  documents into the Qdrant vector database in the 1st collection
parent_retriever.add_documents(docs)

# testing the retriever
parent_retriever.invoke("How many credit hours is a major in Computer Science?")


Now for the 2nd method using the Semantic Text Splitter which splits the text based on the meaning within each sentence for more granular control of retrieval.

For this one, we will use the Ensemble Retriever which allows us to combine the results of multiple retrievers, giving them different weights. Within the Ensemble retriever we will use: 

- BM25 Retriever which a retrieval method used by search engines
- Base retriever that comes with using the vectorstore as a retriever


In [ ]:
# semantic text splitting method
# do '%pip install langchain_experimental' if needed
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

semantic_text_splitter = SemanticChunker(
    OpenAIEmbeddings(), 
    breakpoint_threshold_type="percentile")

semantic_docs = semantic_text_splitter.split_documents(docs)
print(semantic_docs[0].page_content)
print(len(semantic_docs))


In [11]:
# creating another instance of a vector store with a new collection using the function we made earlier
qdrant_collection_2 = os.environ['QDRANT_COLLECTION_2']
vector_store_2 = create_vectorstore(qdrant_collection_2)

In [ ]:
from langchain.retrievers import EnsembleRetriever, BM25Retriever
# we already imported the Qdrant vector store and OpenAI embeddings in a previous step

bm25_retriever = BM25Retriever.from_documents(semantic_docs)

# vector_store_2.from_documents(semantic_docs, OpenAIEmbeddings())
vector_store_2_retriever = vector_store_2.as_retriever()

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_store_2_retriever], weights=[0.7, 0.3]
)

# adding the semantically split docs into the vector store
vector_store_2_retriever.add_documents(semantic_docs)

ensemble_retriever.invoke("How many credit hours is a major in Computer Science?")

We are going to have a base option that splits the text using just the RecursiveCharacterTextSplitter like the [original repo](https://github.com/yawbtng/SMUChatBot_Project/blob/main/main.py) does. From there, we're creating a third vector store collection to upload this text into using the vector store as the retriver.

In [ ]:
base_text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, 
                                                length_function=len, add_start_index=True)  
normal_split_docs = base_text_splitter.split_documents(docs)

# checking result
print(normal_split_docs[0].page_content)
print(len(normal_split_docs))

In [ ]:

# getting the collection name of the third vector store
qdrant_collection_0 = os.environ['QDRANT_COLLECTION_0']

# creating the third vector store and retriever
vector_store_0 = create_vectorstore(qdrant_collection_0)
vector_store_0_retriever = vector_store_0.as_retriever()

# adding the recursively split docs into the vector store
vector_store_0_retriever.add_documents(normal_split_docs)

# testing the retriever
vector_store_0_retriever.invoke("How many credit hours is a major in Computer Science?")

## CSV Breakdown
Now we are going to work on uploading the excel file of 115 FAQs into each vector database collection. There are multiple sheets within the excel file so we're going to have to turn each sheet into a CSV using the pandas library and then use langchain's CSV loader to turn them into langchain documents.

The CSVs you see there now were created by iterating through the xlsx file so you can delete those if you want to see them being recreated (be careful not to delete the xlsx file)

In [ ]:
# using the pandas library to work with the excel file and convert it to a data frame
import pandas as pd

# Load the Excel file
excel_path = 'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/SMU FAQs.xlsx'
xlsx = pd.ExcelFile(excel_path)

# checking to see if loading the file worked
print(xlsx.sheet_names)

# Iterate through each sheet and save as a CSV file
csv_files = []
for sheet_name in xlsx.sheet_names:
    # Read the entire sheet to extract the metadata from cell A1
    sheet_df = pd.read_excel(xlsx, sheet_name=sheet_name, header=None)
    
    # getting the link of the webpage to include as the metadata 
    metadata = sheet_df.iat[0, 0]
    
    # Read the sheet into a DataFrame starting from the second row
    df = pd.read_excel(xlsx, sheet_name=sheet_name, skiprows=1)
    
    # Save the DataFrame to a CSV file
    csv_path = f'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/{sheet_name}.csv'
    df.to_csv(csv_path, index=False)
    csv_files.append((csv_path, metadata))

# Display the list of generated CSV files and their metadata
csv_files

In [ ]:
# Now turning each csv into a langchain document
from langchain.document_loaders import CSVLoader

# Create LangChain documents from CSV files with metadata
csv_documents = []

for csv_path, metadata in csv_files:
    loader = CSVLoader(file_path=csv_path)
    csv_docs = loader.load()
    for csv_doc in csv_docs:
        csv_doc.metadata['source'] = metadata
    csv_documents.extend(csv_docs)

# Display the first document as an example
print(csv_documents[0])

Then lastly upload the csv documents into each vector store collection

In [17]:
# vector store collection 1 - uses parent/child text splitter with parent retriever
parent_retriever.add_documents(csv_documents)

In [ ]:
# vector store collection 2 - uses semantic text splitter (or chunker) with the ensemble retriever (BM25 + vector store as retriever)
# uploaded to vector store using vector store as the retriever
vector_store_2_retriever.add_documents(csv_documents)

In [ ]:
# vector stoer collection 0 - uses the recursive chatacter text splitter with vector store as the retriever
# base option from last project
vector_store_0_retriever.add_documents(csv_documents)

The last thing we need to do is to create functions for variables we will need in other notebooks or python scripts or notebooks.

In [ ]:
def get_all_langchain_docs():
  return {
    "pdf_docs": docs,
    "csv_docs": csv_documents
  }

def get_all_vectorstores():
  return {
      "vector_store_0": vector_store_0, # collection smu-data_0
      "vector_store_1": vector_store_1, # collection smu-data_1
      "vector_store_2": vector_store_2, # collection smu-data_2
  }

def get_all_retrievers():
  return {
      "vector_store_0_retriever": vector_store_0_retriever, # collection smu-data_0
      "parent_retriever": parent_retriever, # collection smu-data_1
      "ensemble_retriever": ensemble_retriever, # collection smu-data_2
  }

Now we are officially done with the data-preprocessing step and can move on to the RAG pipelines for each model!